In [2]:
# Cài đặt thư viện
!pip install -q langchain langchain-community langchain-core neo4j transformers accelerate bitsandbytes torch pydantic

In [ ]:
# Import libraries
import json
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.graphs import Neo4jGraph
from tqdm import tqdm
import time
from pydantic import BaseModel, Field
from typing import List

In [ ]:
# Setup Neo4j connection
NEO4J_URI = "neo4j+s://0c367113.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gTO1K567hBLzkRdUAhhEb-UqvBjz0i3ckV3M9v_-Nio"

graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [ ]:
chunk_file = "/home/duo/work/DL/btl/data/chunk/chunks_data.jsonl"

# Load JSONL file (mỗi dòng là một JSON object)
chunks = []
with open(chunk_file, 'r', encoding='utf-8') as f:
    for line in f:
        chunk_data = json.loads(line.strip())
        chunks.append(chunk_data)

print(f"Đã load {len(chunks)} chunks từ file luật giao thông")
print(f"Ví dụ chunk đầu tiên: {chunks[0]['id']}")
print(f"Nội dung: {chunks[0]['content'][:200]}...")

In [ ]:
# Load Qwen2.5 model từ Hugging Face
print("Đang load model Qwen2.5-7B-Instruct...")

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True
)


In [ ]:
# Define Pydantic schemas cho validation
class Entity(BaseModel):
    """Schema cho một entity"""
    name: str = Field(description="Tên thực thể")
    type: str = Field(description="Loại thực thể: VEHICLE (phương tiện), REGULATION (quy định), PENALTY (mức phạt), ROAD_ELEMENT (yếu tố đường), VIOLATION (vi phạm), ORGANIZATION (cơ quan)")
    description: str = Field(description="Mô tả về thực thể")

class Relationship(BaseModel):
    """Schema cho một relationship"""
    source: str = Field(description="Tên entity nguồn")
    target: str = Field(description="Tên entity đích")
    type: str = Field(description="Loại quan hệ")
    description: str = Field(description="Mô tả quan hệ")

class KnowledgeGraph(BaseModel):
    """Schema cho toàn bộ knowledge graph output"""
    entities: List[Entity] = Field(default_factory=list, description="Danh sách entities")
    relationships: List[Relationship] = Field(default_factory=list, description="Danh sách relationships")

print("Pydantic schemas đã sẵn sàng!")
print(f"\nSchema: {KnowledgeGraph.model_json_schema()}")

In [ ]:
# Prompt template để extract entities và relationships
EXTRACTION_PROMPT = """Phân tích văn bản luật giao thông đường bộ Việt Nam sau và trích xuất entities và relationships.

QUY TẮC:
- Chỉ trích xuất thông tin CÓ TRONG văn bản
- Entity types: 
  + VEHICLE: phương tiện giao thông (xe máy, ô tô, xe thô sơ...)
  + REGULATION: quy định, điều luật
  + PENALTY: mức phạt, hình thức xử lý
  + ROAD_ELEMENT: phần đường, làn đường, vỉa hè, biển báo...
  + VIOLATION: hành vi vi phạm
  + ORGANIZATION: cơ quan quản lý (Cảnh sát giao thông, Bộ GTVT...)
- Relationship phải rõ ràng và có trong văn bản

VĂN BẢN:
{text}

Hãy trả về kết quả theo ĐÚNG định dạng JSON sau (ĐẢM BẢO cú pháp JSON hợp lệ):
{{
  "entities": [
    {{"name": "Xe máy", "type": "VEHICLE", "description": "Phương tiện giao thông cơ giới đường bộ"}},
    {{"name": "Vượt đèn đỏ", "type": "VIOLATION", "description": "Hành vi vi phạm tín hiệu đèn giao thông"}},
    {{"name": "Phạt tiền từ 4-6 triệu đồng", "type": "PENALTY", "description": "Mức phạt vi phạm đèn tín hiệu"}}
  ],
  "relationships": [
    {{"source": "Xe máy", "target": "Vượt đèn đỏ", "type": "CÓ_THỂ_VI_PHẠM", "description": "Xe máy có thể vi phạm vượt đèn đỏ"}},
    {{"source": "Vượt đèn đỏ", "target": "Phạt tiền từ 4-6 triệu đồng", "type": "BỊ_PHẠT", "description": "Vi phạm đèn đỏ bị phạt tiền"}}
  ]
}}

CHỈ TRẢ VỀ JSON, KHÔNG GIẢI THÍCH THÊM:"""

def generate_structured(prompt: str) -> str:
    """Generate text với Qwen model"""
    messages = [
        {"role": "system", "content": "Bạn là một chuyên gia phân tích văn bản luật giao thông đường bộ Việt Nam. Trả về kết quả dưới dạng JSON hợp lệ."},
        {"role": "user", "content": prompt}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=1024,  # Giảm để nhanh hơn
            temperature=0.1,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.1
        )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
def extract_knowledge(text: str, max_retries: int = 3) -> dict:
    """Extract entities và relationships từ text - validate với Pydantic, có retry khi lỗi"""
    
    for attempt in range(max_retries):
        try:
            # Tạo prompt
            prompt = EXTRACTION_PROMPT.format(text=text)
            
            # Generate response
            response = generate_structured(prompt)
            
            # Parse JSON từ response
            json_match = re.search(r'\{.*\}', response, re.DOTALL)
            if json_match:
                json_str = json_match.group(0)
                data = json.loads(json_str)
                
                # Validate với Pydantic
                kg = KnowledgeGraph(**data)
                
                # Convert to dict
                result = {
                    "entities": [e.model_dump() for e in kg.entities],
                    "relationships": [r.model_dump() for r in kg.relationships]
                }
                
                # Thành công
                if attempt > 0:
                    print(f"Retry thành công sau {attempt + 1} lần thử")
                return result
            else:
                print(f"Không tìm thấy JSON (lần {attempt + 1}/{max_retries})")
                if attempt == max_retries - 1:
                    return {"entities": [], "relationships": []}
                continue
            
        except json.JSONDecodeError as e:
            print(f"JSON decode error (lần {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                return {"entities": [], "relationships": []}
            continue
            
        except Exception as e:
            print(f"Error (lần {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                return {"entities": [], "relationships": []}
            continue
    
    return {"entities": [], "relationships": []}

In [ ]:
# Hàm thêm entities và relationships vào Neo4j (tối ưu cho luật giao thông)
def add_to_graph(knowledge: dict, chunk_metadata: dict):
    """Thêm knowledge vào Neo4j graph với metadata luật giao thông"""
    
    # Extract metadata
    law_code = chunk_metadata.get('law_code', 'UNKNOWN')
    law_name = chunk_metadata.get('law_name', '')
    article = chunk_metadata.get('article', 0)
    chapter = chunk_metadata.get('chapter', '')
    chapter_title = chunk_metadata.get('chapter_title', '')
    mode = chunk_metadata.get('mode', '')
    has_penalty = chunk_metadata.get('has_penalty', False)
    vehicles = chunk_metadata.get('vehicles', [])
    
    # 1. Thêm entities với metadata đầy đủ
    for entity in knowledge.get('entities', []):
        name = entity.get('name', '').strip()
        entity_type = entity.get('type', 'UNKNOWN')
        description = entity.get('description', '')
        
        if not name or len(name) < 2:  # Tránh entities quá ngắn
            continue
        
        # Query tối ưu với index và properties phù hợp
        query = f"""
        MERGE (e:{entity_type} {{name: $name}})
        ON CREATE SET 
            e.description = $description,
            e.law_code = $law_code,
            e.law_name = $law_name,
            e.article = $article,
            e.chapter = $chapter,
            e.mode = $mode,
            e.created_at = datetime()
        ON MATCH SET
            e.description = CASE 
                WHEN e.description IS NULL OR e.description = '' 
                THEN $description 
                ELSE e.description 
            END,
            e.last_seen_article = $article,
            e.last_updated = datetime()
        RETURN e.name as name
        """
        
        try:
            graph.query(query, {
                'name': name,
                'description': description,
                'law_code': law_code,
                'law_name': law_name,
                'article': article,
                'chapter': chapter,
                'mode': mode
            })
        except Exception as e:
            print(f"Error adding entity {name}: {e}")
    
    # 2. Thêm relationships với context
    for rel in knowledge.get('relationships', []):
        source = rel.get('source', '').strip()
        target = rel.get('target', '').strip()
        rel_type = rel.get('type', 'RELATED_TO').replace(' ', '_').replace('-', '_').upper()
        description = rel.get('description', '')
        
        if not source or not target or len(source) < 2 or len(target) < 2:
            continue
        
        # Query tối ưu với MERGE relationship
        query = f"""
        MATCH (s {{name: $source}})
        MATCH (t {{name: $target}})
        MERGE (s)-[r:{rel_type}]->(t)
        ON CREATE SET 
            r.description = $description,
            r.law_code = $law_code,
            r.article = $article,
            r.has_penalty = $has_penalty,
            r.created_at = datetime()
        ON MATCH SET
            r.last_seen_article = $article,
            r.occurrence_count = COALESCE(r.occurrence_count, 0) + 1
        RETURN type(r) as rel_type
        """
        
        try:
            graph.query(query, {
                'source': source,
                'target': target,
                'description': description,
                'law_code': law_code,
                'article': article,
                'has_penalty': has_penalty
            })
        except Exception as e:
            # Nếu không tìm thấy entity, bỏ qua (có thể entity bị filter)
            pass
    
    # 3. Liên kết vehicles với entities nếu có
    if vehicles and len(vehicles) > 0:
        for vehicle in vehicles:
            vehicle_name = vehicle.strip()
            if not vehicle_name or len(vehicle_name) < 2:
                continue
                
            query = """
            MERGE (v:VEHICLE {name: $vehicle_name})
            ON CREATE SET v.mode = $mode
            WITH v
            MATCH (e)
            WHERE e.article = $article AND e.law_code = $law_code
            MERGE (v)-[r:MENTIONED_IN]->(e)
            ON CREATE SET r.article = $article
            """
            
            try:
                graph.query(query, {
                    'vehicle_name': vehicle_name,
                    'mode': mode,
                    'article': article,
                    'law_code': law_code
                })
            except Exception as e:
                pass


In [ ]:
# Xóa dữ liệu cũ
clear_old_data = False  # Đổi thành True nếu muốn xóa

if clear_old_data:
    graph.query("MATCH (n) DETACH DELETE n")
    print("Đã xóa dữ liệu cũ")
else:
    print("Giữ nguyên dữ liệu cũ")

In [ ]:
# Tạo indexes để tối ưu hiệu suất query
print("Đang tạo indexes cho Neo4j...")

indexes = [
    "CREATE INDEX vehicle_name IF NOT EXISTS FOR (v:VEHICLE) ON (v.name)",
    "CREATE INDEX violation_name IF NOT EXISTS FOR (v:VIOLATION) ON (v.name)",
    "CREATE INDEX penalty_name IF NOT EXISTS FOR (p:PENALTY) ON (p.name)",
    "CREATE INDEX regulation_name IF NOT EXISTS FOR (r:REGULATION) ON (r.name)",
    "CREATE INDEX road_element_name IF NOT EXISTS FOR (r:ROAD_ELEMENT) ON (r.name)",
    "CREATE INDEX organization_name IF NOT EXISTS FOR (o:ORGANIZATION) ON (o.name)",
]

for idx_query in indexes:
    try:
        graph.query(idx_query)
        print(f"  ✓ Đã tạo index")
    except Exception as e:
        print(f"  ! Index có thể đã tồn tại: {str(e)[:50]}")

print("\nIndexes đã sẵn sàng!")

In [ ]:
# Process only chunks from start_chunk to end_chunk
start_chunk = 0  
end_chunk = 30  
total_entities = 0
total_relationships = 0
errors = 0

num_chunks_to_process = min(end_chunk, len(chunks)) - start_chunk
print(f"Bắt đầu xử lý chunks từ {start_chunk} đến {end_chunk-1} (tổng {num_chunks_to_process})...")
for i in tqdm(range(start_chunk, min(end_chunk, len(chunks))), desc="Processing chunks"):
    chunk = chunks[i]
    
    try:
        # Extract knowledge
        knowledge = extract_knowledge(chunk['content'])
        
        # Add to graph
        add_to_graph(knowledge, chunk['metadata'])
        
        # Stats
        entities_count = len(knowledge.get('entities', []))
        rels_count = len(knowledge.get('relationships', []))
        total_entities += entities_count
        total_relationships += rels_count
        
        # Print progress
        if ((i - start_chunk + 1) % 5) == 0:
            print(f"\nProgress after {i-start_chunk+1} chunks:")
            print(f"   Entities: {total_entities}")
            print(f"   Relationships: {total_relationships}")
            print(f"   Errors: {errors}")
        
    except Exception as e:
        errors += 1
        print(f"\nError at chunk {i}: {e}")
        continue

print(f"HOÀN TẤT!")
print(f"Thống kê:")
print(f"   - Chunks xử lý: {num_chunks_to_process}")
print(f"   - Tổng entities: {total_entities}")
print(f"   - Tổng relationships: {total_relationships}")
print(f"   - Errors: {errors}")
print(f"   - Success rate: {((num_chunks_to_process - errors) / num_chunks_to_process * 100):.1f}%")

In [ ]:
# Kiểm tra kết quả trong Neo4j (tối ưu cho luật giao thông)
print("=== Thống kê Entities theo loại ===")
count_query = """
MATCH (n)
RETURN labels(n)[0] as Type, count(n) as Count
ORDER BY Count DESC
"""
results = graph.query(count_query)
total_entities = 0
for row in results:
    print(f"  {row['Type']}: {row['Count']}")
    total_entities += row['Count']
print(f"\nTổng: {total_entities} entities")

print("\n=== Thống kê Relationships ===")
rel_count_query = """
MATCH ()-[r]->()
RETURN type(r) as RelType, count(r) as Count
ORDER BY Count DESC
LIMIT 20
"""
rel_results = graph.query(rel_count_query)
total_rels = 0
for row in rel_results:
    print(f"  {row['RelType']}: {row['Count']}")
    total_rels += row['Count']
print(f"\nTổng relationships (top 20): {total_rels}")

print("\n=== Thống kê theo Luật ===")
law_query = """
MATCH (n)
WHERE n.law_code IS NOT NULL
RETURN n.law_code as LawCode, n.law_name as LawName, count(n) as Count
ORDER BY Count DESC
LIMIT 10
"""
law_results = graph.query(law_query)
for row in law_results:
    print(f"  {row['LawCode']}: {row['Count']} entities")

print("\n=== Thống kê Entities có mức phạt ===")
penalty_query = """
MATCH (p:PENALTY)
RETURN count(p) as PenaltyCount
"""
penalty_results = graph.query(penalty_query)
for row in penalty_results:
    print(f"  Tổng mức phạt: {row['PenaltyCount']}")

print("\n=== Thống kê Vi phạm ===")
violation_query = """
MATCH (v:VIOLATION)
OPTIONAL MATCH (v)-[r:BỊ_PHẠT]->(p:PENALTY)
RETURN count(v) as ViolationCount, count(r) as WithPenalty
"""
violation_results = graph.query(violation_query)
for row in violation_results:
    print(f"  Tổng vi phạm: {row['ViolationCount']}")
    print(f"  Vi phạm có mức phạt: {row['WithPenalty']}")

In [ ]:
# Phân tích chi tiết các entities quan trọng
print("=== Top 15 Vi phạm phổ biến nhất ===")
violation_query = """
MATCH (v:VIOLATION)
OPTIONAL MATCH (v)-[r]-()
WITH v, count(DISTINCT r) as rel_count
RETURN v.name as Violation, v.description as Description, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 15
"""
violations = graph.query(violation_query)
for i, row in enumerate(violations, 1):
    print(f"{i:2d}. {row['Violation']} - {row['Connections']} kết nối")

print("\n=== Top 10 Phương tiện được đề cập ===")
vehicle_query = """
MATCH (v:VEHICLE)
OPTIONAL MATCH (v)-[r]-()
WITH v, count(DISTINCT r) as rel_count
RETURN v.name as Vehicle, v.mode as Mode, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 10
"""
vehicles = graph.query(vehicle_query)
for i, row in enumerate(vehicles, 1):
    mode = row.get('Mode', 'N/A')
    print(f"{i:2d}. {row['Vehicle']} ({mode}) - {row['Connections']} kết nối")

print("\n=== Top 10 Mức phạt ===")
penalty_query = """
MATCH (p:PENALTY)
OPTIONAL MATCH (v)-[r:BỊ_PHẠT]->(p)
WITH p, count(v) as violation_count
RETURN p.name as Penalty, p.description as Description, violation_count as Violations
ORDER BY violation_count DESC
LIMIT 10
"""
penalties = graph.query(penalty_query)
for i, row in enumerate(penalties, 1):
    desc = row.get('Description', '')[:50]
    vio_count = row.get('Violations', 0)
    print(f"{i:2d}. {row['Penalty']}")
    print(f"     Vi phạm liên quan: {vio_count}")

print("\n=== Các quy định quan trọng ===")
regulation_query = """
MATCH (r:REGULATION)
OPTIONAL MATCH (r)-[rel]-()
WITH r, count(DISTINCT rel) as rel_count
WHERE rel_count > 0
RETURN r.name as Regulation, r.article as Article, rel_count as Connections
ORDER BY rel_count DESC
LIMIT 10
"""
regulations = graph.query(regulation_query)
for i, row in enumerate(regulations, 1):
    article = row.get('Article', 'N/A')
    print(f"{i:2d}. {row['Regulation']} (Điều {article}) - {row['Connections']} kết nối")